# Trabalho Programação Linear 

## Leitura dos dados 

In [2]:
from pandas_ods_reader import read_ods

df = read_ods("variaveis_decisao.ods", 1)
df

,Item,Unidade,Local transporte,Variável de decisão,Volume [cm^3],Venda (R$),Compra,Lucro,Demanda
0,Ovo de codorna,Dúzia,Kombi,x1,560.0,5.00,3.00,2.00,180.0
1,Ovo Branco,Dúzia,Kombi,x2,1458.0,5.00,3.50,1.50,90.0
2,Ovo Vermelho,Dúzia,Kombi,x3,1458.0,7.00,5.00,2.00,60.0
3,Coxa de Frango,Embalagem,Geladeira,x4,640.0,12.90,7.00,5.90,24.0
4,Picanha Suína,Embalagem,Geladeira,x5,630.0,26.90,18.80,8.10,6.0
5,Picanha Bovina,Embalagem,Geladeira,x6,1512.0,49.40,36.40,13.00,7.0
6,Miúdo de Porco,Embalagem,Geladeira,x7,13950.0,38.70,24.00,14.70,7.0
7,Pé de Frango,Caixa,Geladeira,x8,112404.0,85.00,34.00,51.00,36.0
8,Carré,Embalagem,Geladeira,x9,20160.0,81.12,48.00,33.12,10.0
9,Carne Moída,Embalagem,Geladeira,x10,15876.0,48.00,40.00,8.00,6.0


## Definição das variáveis 

In [3]:
volume = list(df['Volume [cm^3]'])
preco_venda = list(df['Venda (R$)'])
preco_compra = list(df['Compra'])
lucro = list(df['Lucro'])
demanda = list(df["Demanda"])

## Definição das matrizes de Entrada 

### Função Objetiva

Iremos minimizar a -f(x1, ... , x22), onde f(x1, ... , x22) é a função que representa o valor total da venda.

In [4]:
Z = [x * -1 for x in preco_venda]

### Restrições

Nesta etapa iremos impor as restrições das variáves.

#### Restrição Geladeira

Nesta etapa selecionaremos os itens que irão dentro da geladeira, ou seja, de x4 a x22.

In [5]:
aux = df.copy()
i = 0
for item in aux['Local transporte']:
    if item != 'Geladeira':
        aux.loc[i, 'Volume [cm^3]'] = 0
    i += 1
G = list(aux['Volume [cm^3]'])
max_G = 1369900

#### Restrição Kombi

Agora iremos impor as restrições para os itens que irão fora da geladeira. 

In [6]:
aux = df.copy()
i = 0
for item in aux['Local transporte']:
    if item == 'Geladeira':
        aux.loc[i, 'Volume [cm^3]'] = 0
    i += 1
K = list(aux['Volume [cm^3]'])
max_K = 1984000

#### Fluxo de caixa

Existe um limite diário de mercadória de R$ 6000,00.

In [7]:
F = [x * -1 for x in preco_venda]
max_F = 6000

# Aplicando o Simplex 

In [11]:
import scipy
from scipy.optimize import linprog

In [10]:
c = Z
A = [G, K, F]
B = [max_G, max_K, max_F]
x1_bd = (0, None)
x_bounds = (x1_bd,)*len(c)

In [12]:
modelo_simplex = linprog(c, A_ub=A, b_ub=B, bounds=x_bounds, method='Simplex', options={"disp":True})

Optimization terminated successfully.
         Current function value: -76206.841270
         Iterations: 9


In [13]:
modelo_simplex

     con: array([], dtype=float64)
     fun: -76206.84126984127
 message: 'Optimization terminated successfully.'
     nit: 9
   slack: array([ 0.00000000e+00, -2.32830644e-10,  8.22068413e+04])
  status: 0
 success: True
       x: array([3542.85714286,    0.        ,    0.        ,    0.        ,
       2174.44444444,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ])

In [79]:
c = Z
A = [G, K, F]
B = [max_G, max_K, max_F]
x_bounds = []
for item in demanda:
    x_bounds.append((0.8*item,1.2*item))

x_bounds = tuple(x_bounds)
x_bounds

((144.0, 216.0),
 (72.0, 108.0),
 (48.0, 72.0),
 (19.200000000000003, 28.799999999999997),
 (4.800000000000001, 7.199999999999999),
 (5.6000000000000005, 8.4),
 (5.6000000000000005, 8.4),
 (28.8, 43.199999999999996),
 (8.0, 12.0),
 (4.800000000000001, 7.199999999999999),
 (9.600000000000001, 14.399999999999999),
 (3.2, 4.8),
 (4.0, 6.0),
 (8.0, 12.0),
 (19.200000000000003, 28.799999999999997),
 (5.6000000000000005, 8.4),
 (12.0, 18.0),
 (38.400000000000006, 57.599999999999994),
 (3.2, 4.8),
 (4.0, 6.0),
 (4.0, 6.0),
 (4.0, 6.0))

In [141]:
model_simplex = linprog(c, A_ub=A, b_ub=B, bounds=x_bounds, method='revised simplex', options={ 'disp': False, 'tol':4e-01})
model_simplex

     con: array([], dtype=float64)
     fun: -11467.816
 message: 'The problem appears infeasible, as the phase one auxiliary problem terminated successfully with a residual of 3.7e+06, greater than the tolerance 0.4 required for the solution to be considered feasible. Consider increasing the tolerance to be greater than 3.7e+06. If this tolerance is unnaceptably large, the problem is likely infeasible.'
     nit: 0
   slack: array([-3652845.6  ,  1370488.   ,    17467.816])
  status: 2
 success: False
       x: array([144. ,  72. ,  48. ,  19.2,   4.8,   5.6,   5.6,  28.8,   8. ,
         4.8,   9.6,   3.2,   4. ,   8. ,  19.2,   5.6,  12. ,  38.4,
         3.2,   4. ,   4. ,   4. ])

In [148]:
pi = linprog(c, A_ub=A, b_ub=B, bounds=x_bounds, method='interior-point', options={"disp":True})

Primal Feasibility  Dual Feasibility    Duality Gap         Step             Path Parameter      Objective          
1.0                 1.0                 1.0                 -                1.0                 -12780.826          
0.01584399343998    0.01584399343998    0.01584399343998    0.997398071718   0.01584399343998    -12577.44616061     
0.006028663062925   0.006028663062925   0.006028663062925   0.6335973194192  0.006028663062926   -12668.73220103     
0.002181583372656   0.002181583372656   0.002181583372656   0.6598317368356  0.002181583372656   -12552.8068186      
0.00167036610939    0.00167036610939    0.001670366109389   0.2589492125655  0.00167036610939    -12712.55122754     
1.352568823305e-05  1.352568823299e-05  1.35256882367e-05   0.9932819289547  1.352568823305e-05  -12187.78593884     
2.206915781734e-07  2.206915781653e-07  2.206915809757e-07  0.987748397945   2.206915781734e-07  -12725.4052152      
4.605468711694e-11  4.605467614022e-11  4.605003278954e-1

In [149]:
pi

     con: array([], dtype=float64)
     fun: -12077.126892382772
 message: 'The algorithm terminated successfully and determined that the problem is infeasible.'
     nit: 7
   slack: array([-3744915.17031751,  1337043.23696892,    18077.12689238])
  status: 2
 success: False
       x: array([146.60690355,  73.64450861,  49.40656139,  20.37836095,
         5.96737858,   6.86490694,   5.89207318,  29.14803045,
         8.18779693,   5.05772395,   9.73604575,   3.72836299,
         5.10283773,   8.60360518,  19.65632135,   6.64783524,
        12.10703929,  38.6766147 ,   3.54515468,   4.29478946,
         4.29478946,   4.44284383])

# Segunda Tentativa 

In [153]:
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable,pulpTestAll, GLPK

In [135]:
x1 = LpVariable(name="x1", lowBound=144, upBound = 216, cat="Integer")
x2 = LpVariable(name="x2", lowBound=72, upBound = 108, cat="Integer")
x3 = LpVariable(name="x3", lowBound=48, upBound = 72, cat="Integer")
x4 = LpVariable(name="x4", lowBound=19.2, upBound = 28.8, cat="Integer")
x5 = LpVariable(name="x5", lowBound=4.8, upBound = 7.2, cat="Integer")
x6 = LpVariable(name="x6", lowBound=5.6, upBound = 8.4, cat="Integer")
x7 = LpVariable(name="x7", lowBound=5.6, upBound = 8.4, cat="Integer")
x8 = LpVariable(name="x8",  lowBound=28.8, upBound = 43.2, cat="Integer")
x9 = LpVariable(name="x9",  lowBound=8, upBound = 12, cat="Integer")
x10 = LpVariable(name="x10", lowBound=4.8, upBound =7.2, cat="Integer")
x11 = LpVariable(name="x11",  lowBound=9.6, upBound =14.4,cat="Integer")
x12 = LpVariable(name="x12",  lowBound=3.2, upBound =4.8, cat="Integer")
x13 = LpVariable(name="x13", lowBound=4, upBound = 6, cat="Integer")
x14 = LpVariable(name="x14",   lowBound=8, upBound = 12, cat="Integer")
x15 = LpVariable(name="x15", lowBound=19.2, upBound = 28.8, cat="Integer")
x16 = LpVariable(name="x16",  lowBound=5.2, upBound = 8.4, cat="Integer")
x17 = LpVariable(name="x17",  lowBound=12, upBound = 8, cat="Integer")
x18 = LpVariable(name="x18",  lowBound=38.4, upBound = 57.6, cat="Integer")
x19 = LpVariable(name="x19",  lowBound=3.2, upBound = 4.8, cat="Integer")
x20 = LpVariable(name="x20", lowBound=4, upBound = 6, cat="Integer")
x21 = LpVariable(name="x21",  lowBound=4, upBound = 6, cat="Integer")
x22 = LpVariable(name="x22",  lowBound=4, upBound = 6, cat="Integer")

In [228]:
x1 = LpVariable(name="x1", lowBound=144, cat="Integer")
x2 = LpVariable(name="x2", lowBound=72, cat="Integer")
x3 = LpVariable(name="x3", lowBound=0, cat="Integer")
x4 = LpVariable(name="x4", lowBound=0, cat="Integer")
x5 = LpVariable(name="x5", lowBound=0, cat="Integer")
x6 = LpVariable(name="x6", lowBound=0, cat="Integer")
x7 = LpVariable(name="x7", lowBound=0, cat="Integer")
x8 = LpVariable(name="x8",  lowBound=0, cat="Integer")
x9 = LpVariable(name="x9",  lowBound=0, cat="Integer")
x10 = LpVariable(name="x10", lowBound=0, cat="Integer")
x11 = LpVariable(name="x11",  lowBound=0,cat="Integer")
x12 = LpVariable(name="x12",  lowBound=0, cat="Integer")
x13 = LpVariable(name="x13", lowBound=0, cat="Integer")
x14 = LpVariable(name="x14",   lowBound=0, cat="Integer")
x15 = LpVariable(name="x15", lowBound=0, cat="Integer")
x16 = LpVariable(name="x16",  lowBound=0, cat="Integer")
x17 = LpVariable(name="x17",  lowBound=0, cat="Integer")
x18 = LpVariable(name="x18",  lowBound=0, cat="Integer")
x19 = LpVariable(name="x19",  lowBound=0, cat="Integer")
x20 = LpVariable(name="x20", lowBound=0, cat="Integer")
x21 = LpVariable(name="x21",  lowBound=0, cat="Integer")
x22 = LpVariable(name="x22",  lowBound=0, cat="Integer")

In [229]:
# Add the constraints to the model
# Create the model
model = LpProblem(name="small-problem", sense=LpMaximize)

model += (0*x1+0*x2+0*x3+640*x4+630*x5+1512*x6+13950*x7+112404*x8+20160*x9+
          15876*x10+30618*x11+5985*x12+560*x13+4851*x14+7942*x15+1206*x16+33840*x17+11232*x18
          + 10800*x19+ 0*x20 + 0*x21+ 0*x22 <= 1369900)

model += (560*x1+1458*x2+1458*x3+0*x4+0*x5+0*x6+0*x7+0*x8+0*x9+
          0*x10+0*x11+0*x12+0*x13+0*x14+0*x15+0*x16+0*x17+0*x18
          +0*x19+ 40824*x20 + 40824*x21 +7830*x22<= 1984000)

model += (3*x1+3.5*x2+5*x3+7*x4+18.8*x5+
          36.4*x6+24*x7+34*x8+48*x9+40*x10+
          24*x11+3*x12+16*x13+2*x14+14*x15+
          113.4*x16+
          70*x17+8*x18+5.56*x19+ 200*x20 + 
          200*x21 +10*x22<= 6000)

model += lpSum([5*x1,5*x2,7*x3,12.9*x4,26.9*x5,
            49.4*x6,38.7*x7,85*x8,81.12*x9,
          48*x10,39.6*x11,5*x12,19*x13,5*x14,25.8*x15,
            156.87*x16,90*x17,12.9*x18
          , 7.92*x19, 289*x20 , 289*x21, 13.9*x22])

In [230]:
status = model.solve()

In [231]:
print(f"status: {model.status}, {LpStatus[model.status]}")

status: 1, Optimal


In [157]:
print(f"objective: {model.objective.value()}")

objective: 195784.35


In [232]:
for var in model.variables():
    print(f"{var.name}: {var.value()}")

x1: 145.0
x10: 0.0
x11: 0.0
x12: 0.0
x13: 0.0
x14: 189.0
x15: 0.0
x16: 0.0
x17: 0.0
x18: 0.0
x19: 0.0
x2: 72.0
x20: 0.0
x21: 0.0
x22: 0.0
x3: 0.0
x4: 705.0
x5: 0.0
x6: 0.0
x7: 0.0
x8: 0.0
x9: 0.0


In [134]:
for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.value()}")

_C1: -460.0
_C2: -480.0
_C3: -1844646.2
